# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 64 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.00009


extracting tarball to tmp_2302.00009...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.00012


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2302.00009/natbib.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

Retrieving document from  https://arxiv.org/e-print/2302.00497


extracting tarball to tmp_2302.00497... done.
Retrieving document from  https://arxiv.org/e-print/2302.00531


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2302.00497/data_center.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2302.00531...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00450-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00450) | **Sub-kiloparsec empirical relations and excitation conditions of HCN and  HCO+ J=3-2 in nearby star-forming galaxies**  |
|| Axel Garcia-Rodriguez, et al. -- incl., <mark>Frank Bigiel</mark>, <mark>Daizhong Liu</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2023-02-02*|
|*Comments*| *Accepted for publication in A&A. 14 pages, 8 figures*|
|**Abstract**| We present new HCN and HCO$^+$ ($J$=3-2) images of the nearby star-forming galaxies (SFGs) NGC 3351, NGC 3627, and NGC 4321. The observations, obtained with the Morita ALMA Compact Array, have a spatial resolution of $\sim$290-440 pc and resolve the inner $R_\textrm{gal} \lesssim$ 0.6-1 kpc of the targets, as well as the southern bar end of NGC 3627. We complement this data set with publicly available images of lower excitation lines of HCN, HCO$^+$, and CO and analyse the behaviour of a representative set of line ratios: HCN(3-2)/HCN(1-0), HCN(3-2)/HCO$^+$(3-2), HCN(1-0)/CO(2-1), and HCN(3-2)/CO(2-1). Most of these ratios peak at the galaxy centres and decrease outwards. We compare the HCN and HCO$^+$ observations with a grid of one-phase, non-local thermodynamic equilibrium (non-LTE) radiative transfer models and find them compatible with models that predict subthermally excited and optically thick lines. We study the systematic variations of the line ratios across the targets as a function of the stellar surface density ($\Sigma_\textrm{star}$), the intensity-weighted CO(2-1) ($\langle I_\text{CO}\rangle$), and the star formation rate surface density ($\Sigma_\text{SFR}$). We find no apparent correlation with $\Sigma_\text{SFR}$, but positive correlations with the other two parameters, which are stronger in the case of $\langle I_\text{CO}\rangle$. The HCN/CO-$\langle I_\text{CO}\rangle$ relations show $\lesssim$0.3 dex galaxy-to-galaxy offsets, with HCN(3-2)/CO(2-1)-$\langle I_\text{CO}\rangle$ being $\sim$2 times steeper than HCN(1-0)/CO(2-1). In contrast, the HCN(3-2)/HCN(1-0)-$\langle I_\text{CO}\rangle$ relation exhibits a tighter alignment between galaxies. We conclude that the overall behaviour of the line ratios cannot be ascribed to variations in a single excitation parameter (e.g. density or temperature). |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00009-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00009) | **Evidence that Core-Powered Mass-Loss Dominates Over Photoevaporation in  Shaping the Kepler Radius Valley**  |
|| Travis A. Berger, Joshua E. Schlieder, <mark>Daniel Huber</mark>, Thomas Barclay |
|*Appeared on*| *2023-02-02*|
|*Comments*| *10 pages, 4 figures. Submitted to ApJL*|
|**Abstract**| The dearth of planets with sizes around 1.8 $\mathrm{R_\oplus}$ is a key demographic feature discovered by the $Kepler$ mission. Two theories have emerged as potential explanations for this valley: photoevaporation and core-powered mass-loss. However, Rogers et al. (2021) shows that differentiating between the two theories is possible using the three-dimensional parameter space of planet radius, incident flux, and stellar mass. We use homogeneously-derived stellar and planetary parameters to measure the $Kepler$ exoplanet radius gap in this three-dimensional space. We compute the slope of the gap as a function of incident flux at constant stellar mass ($\alpha$ $\equiv$ $\left(\partial \log R_{\mathrm{gap}} / \partial \log S \right)_{M_\star}$) and the slope of the gap as a function of stellar mass at constant incident flux ($\beta$ $\equiv$ $\left(\partial \log R_{\mathrm{gap}} / \partial \log M_\star \right)_{S}$) and find $\alpha$ = 0.069$^{+0.019}_{-0.023}$ and $\beta$ = $-$0.046$^{+0.125}_{-0.117}$. Given that Rogers et al. (2021) shows that core-powered mass-loss predicts $\alpha$ $\approx$ 0.08 and $\beta$ $\approx$ 0.00 while photoevaporation predicts $\alpha$ $\approx$ 0.12 and $\beta$ $\approx$ --0.17, our measurements are more consistent with core-powered mass-loss than photoevaporation. However, we caution that different gap-determination methods can produce systematic offsets in both $\alpha$ and $\beta$; therefore, we motivate a comprehensive re-analysis of $Kepler$ light curves with modern, updated priors on eccentricity and mean stellar density to improve both the accuracy and precision of planet radii and subsequent measurements of the gap. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00012-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00012) | **Hidden Little Monsters: Spectroscopic Identification of Low-Mass,  Broad-Line AGN at $z>5$ with CEERS**  |
|| Dale D. Kocevski, et al. -- incl., <mark>Steven L. Finkelstein</mark>, <mark>Linhua Jiang</mark>, <mark>Wenxiu Li</mark>, <mark>Zhengrong Li</mark> |
|*Appeared on*| *2023-02-02*|
|*Comments*| *16 pages, 8 figures, submitted to ApJL*|
|**Abstract**| We report on the discovery of two low-luminosity, broad-line AGN at $z>5$ identified using JWST NIRSpec spectroscopy from the CEERS Survey. We detect broad H$\alpha$ emission from both sources, with FWHM of $2038\pm286$ and $1807\pm207$ km s$^{-1}$, resulting in black hole (BH) masses that are 1-2 dex below that of existing samples of luminous quasars at $z>5$. The first source, CEERS 1670 at $z=5.242$, is 2-3 dex fainter than known quasars at similar redshifts and was previously identified as a candidate low-luminosity AGN based on its rest-frame optical SED. We measure a BH mass of $M_{\rm BH}=1.3\pm0.4\times 10^{7}~M_{\odot}$, confirming that this AGN is powered by the least-massive BH known in the universe at the end of cosmic reionization. The second source, CEERS 3210 at $z=5.624$, is inferred to be a heavily obscured, broad-line AGN caught in a transition phase between a dust-obscured starburst and an unobscured quasar. We estimate its BH mass to be $M_{\rm BH}\simeq 0.9-4.7 \times 10^{7}~M_{\odot}$, depending on the level of dust obscuration assumed. We derive host stellar masses, $M_\star$, allowing us to place constraints on the BH-galaxy mass relationship in the lowest mass range yet probed in the early universe. The $M_{\rm BH}/M_\star$ ratio for CEERS 1670, in particular, is consistent with or higher than the empirical relationship seen in massive galaxies at $z=0$. We examine the emission-line ratios of both sources and find that their location on the BPT and OHNO diagrams is consistent with model predictions for low-metallicity AGN with $Z/Z_\odot \simeq 0.2-0.4$. The spectroscopic identification of low-luminosity, broad-line AGN at $z>5$ with $M_{\rm BH}\simeq 10^{7}~M_{\odot}$ demonstrates the capability of JWST to push BH masses closer to the range predicted for the BH seed population and provides a unique opportunity to study the early stages of BH-galaxy assembly. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00445-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00445) | **Structure and Kinematics of Sh2-138 -- A distant hub-filament system in  the outer Galactic plane**  |
|| <mark>Kshitiz Mallick</mark>, et al. |
|*Appeared on*| *2023-02-02*|
|*Comments*| *16 Pages, 10 Figures, Accepted for publication in The Astrophysical Journal*|
|**Abstract**| We present a molecular line study of the Sh2-138 (IRAS 22308+5812) hub-filament system with an aim to investigate its structure and kinematics. Archival CO molecular line data from the Canadian Galactic Plane Survey (CO(J=1-0)) for the wider region (50arcminx50arcmin) and the James Clerk Maxwell Telescope (CO(3-2), 13CO(3-2), and C18O(3-2)) for the central portion (5arcminx5arcmin) have been utilised. Analysis of the CO(1-0) spectra for the extended region in conjunction with the hub and filament identification using column density map and the getsf tool, respectively, reveals a complex structure with the spectral extraction for the central position displaying multiple velocity components. Based on the Herschel 70 micron warm dust emission, one of the filaments in the extended region was inferred to be associated with active star formation, and is host to a Bolocam 1.1 mm clump of 1606 Msun. Integrated intensity map of 13CO(3-2) emission, constructed from clumps detected at above 5sigma in position-position-velocity space, reveals three filamentary structures (labelled W-f, SW-f, and SE-f) in the central portion. Velocity gradients observed in 13CO(3-2) position-velocity slices point to longitudinal gas flow along the filaments into the central region. Filaments W-f, SW-f, and SE-f were calculated to have observed line masses of 32, 33.5, and 50 Msun/pc , respectively. The cloud was found to be dominated by supersonic and non-thermal motions, with high Mach numbers (>3) and low thermal to non-thermal pressure ratio (0.01-0.1). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00497-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00497) | **The data center for the X-ray spectrometer/imager STIX onboard Solar  Orbiter**  |
|| Hualin Xiao, et al. -- incl., <mark>Olivier Limousin</mark> |
|*Appeared on*| *2023-02-02*|
|*Comments*| *18 pages,submitted to A&A*|
|**Abstract**| The Spectrometer/Telescope for Imaging X-rays (STIX) onboard Solar Orbiter observes solar X-ray emission in the range of 4 to 150 keV and produces spectra and images of solar flares over a wide range of flare magnitudes. During nominal operation, STIX continuously generates data. A constant data flow requires fully automated data processing pipelines to process and analyze and a data platform to manage, visualize, and distribute the data products to the scientific community. The STIX Data Center has been built to fulfill these needs. In this paper, we outline its main components to help the community better understand the tools and data it provides. Methods. The STIX Data Center is operated at the University of Applied Sciences and Arts Northwestern Switzerland (FHNW) and consists of automated processing pipelines and a data platform. The pipelines process STIX telemetry data, perform common analysis tasks and generate data products at different processing levels. They have been designed to operate fully automatically with minimal human intervention. The data platform provides web-based user interfaces and application programmable interfaces for searching and downloading STIX data products. The STIX Data Center has been operating successfully for more than two years. The platform facilitates instrument operations and provides vital support to STIX data users. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00531-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00531) | **Quantifying errors in 3D CME parameters derived from synthetic data  using white-light reconstruction techniques**  |
|| Christine Verbekea, et al. -- incl., <mark>Hebe Cremades</mark> |
|*Appeared on*| *2023-02-02*|
|*Comments*| **|
|**Abstract**| (Shortened version) Current efforts in space weather forecasting of CMEs have been focused on predicting their arrival time and magnetic structure. To make predictions, methods have been developed to derive the true CME speed, size and position, among others. Difficulties in determining input parameters for CME forecasting arise from the lack of direct measurements of the coronal magnetic fields and uncertainties in estimating the CME 3D geometric and kinematic parameters. White-light coronagraph images are usually employed by a variety of CME reconstruction techniques. We explore how subjectivity affects the 3D CME parameters that are obtained from the GCS reconstruction technique. We have designed two different synthetic scenarios where the ``true'' geometric parameters are known in order to quantify such uncertainties for the first time. We explore this as follows: 1) Using the ray-tracing option from the GCS model software, and 2) Using 3D MHD simulation data from the MAS code. Our experiment includes different viewing configurations using single and multiple viewpoints. CME reconstructions using a single viewpoint had the largest errors and error ranges for both synthetic GCS and simulated MHD white-light data. Increasing the number of viewpoints to two, the errors decreased by about 4$^\circ$ in latitude, 22$^\circ$ in longitude, 14$^\circ$ in tilt, and 10$^\circ$ in half-angle, pointing towards a need for at least two viewpoints. We found the following CME parameter error bars as a starting point for quantifying the minimum error in CME parameters from white-light reconstructions: $\Delta\theta$ (latitude)=${6^\circ}$, $\Delta\phi$ (longitude)=${11^\circ}$, $\Delta\gamma$ (tilt)=${25^\circ}$, $\Delta\alpha$ (half-angle)=${10^\circ}$, $\Delta h$ (height)=$0.6$\,$R_{\odot}$, and $\Delta\kappa$ (ratio)=$0.1$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00027-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00027) | **A high-mass X-ray binary descended from an ultra-stripped supernova**  |
|| Noel D. Richardson, et al. -- incl., <mark>Herbert Pablo</mark> |
|*Appeared on*| *2023-02-02*|
|*Comments*| *published in Nature, February 1, 2023*|
|**Abstract**| Ultra-stripped supernovae are different from other terminal explosions of massive stars, as they show little or no ejecta from the actual supernova event. They are thought to occur in massive binary systems after the exploding star has lost its surface through interactions with its companion. Such supernovae produce little to no kick, leading to the formation of a neutron star without loss of the binary companion, which itself may also evolve into another neutron star. Here we show that a recently discovered high-mass X-ray binary, CPD -29 2176 (CD -29 5159; SGR 0755-2933), has an evolutionary history that shows the neutron star component formed during an ultra-stripped supernova. The binary has orbital elements that are similar both in period and in eccentricity to one of 14 Be X-Ray binaries that have both known orbital periods and eccentricities. The identification of the progenitors systems for ultra-stripped supernovae is necessary as their evolution pathways leads to the formation of a binary neutron star systems. Binary neutron stars, such as the system that produced the kilonova GW170817 that was observed with both electromagnetic and gravitational energy, are known to produce a large quantity of heavy elements. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00289-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00289) | **Solar radio bursts observations by Egypt- Alexandria CALLISTO  spectrometer: First results**  |
|| F. N. Minta, et al. -- incl., <mark>A. Mahrous</mark> |
|*Appeared on*| *2023-02-02*|
|*Comments*| *15 pages, 6 figures. Adv Sp Res (2022)*|
|**Abstract**| The newly installed CALLISTO spectrometer, hosted by the Department of Space Environment, Institute of Basic and Applied Sciences- EJUST, commenced operation on August 14, 2021. The system contains a cross dipole long-wavelength array antenna with high sensitivity to monitor solar radio transients. Its antenna was strategically positioned and appeared to be in the center of the CALLISTO network of spectrometers. Moreover, in the northern section of Africa, the Egypt-Alexandria CALLISTO and ALGERIA-CRAAG stations are the only ones operating. There are no stations in the West African region, while stations in the eastern part of Africa are not working. Thus, Egypt- Alexandria station serves as a reference for other stations within the e-CALLISTO network. Despite the low solar activity, the instrument detected several solar radio bursts not limited to type II, type III, and type V. A vigorous case study was conducted on two selected radio burst events to validate the authenticity of the recorded events. Other solar radio stations at different geographical locations recorded all the radio bursts detected by the spectrometer. The case study included brief analyses that indicated a type II radio burst observed on October 09, 2021, between 06:30 and 07:00 UT, was associated with an M1.6 solar flare located at N18E08 within NOAA-AR 12882 and a CME with a shock front speed of ~978 km/s. However, the type III radio burst is neither CME nor solar flare associated. These analyses examine the instrument's capacity to provide real-time solar radio transient data 24 hours a day to mitigate the challenges of data gaps faced in the African continent. Hence, the instrument has become an integral part of space weather monitoring and forecasting over the region and other parts of the globe. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.00450.md
    + _build/html/tmp_2302.00450/./ngc3351-mom0_HCN32.png
    + _build/html/tmp_2302.00450/./ngc3627-mom0_HCN32.png
    + _build/html/tmp_2302.00450/./ngc4321-mom0_HCN32.png
    + _build/html/tmp_2302.00450/./ngc3351-mom0_HCO+32.png
    + _build/html/tmp_2302.00450/./ngc3627-mom0_HCO+32.png
    + _build/html/tmp_2302.00450/./ngc4321-mom0_HCO+32.png
    + _build/html/tmp_2302.00450/./ngc3351-strict_mom0_co21-high_res.png
    + _build/html/tmp_2302.00450/./ngc3627-strict_mom0_co21-high_res.png
    + _build/html/tmp_2302.00450/./ngc4321-strict_mom0_co21-high_res.png
    + _build/html/tmp_2302.00450/./trends_all_galaxies-line_ratios.png
    + _build/html/tmp_2302.00450/./histogram_ratios.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\figcita}[1]{Fig. \ref{#1}}$
$\newcommand{\tabcita}[1]{Table \ref{#1}}$
$\newcommand{\refe}[1]{{\hypersetup{hidelinks}\hyperref[fig_point]{\color{blue}#1}}}$
$\newcommand{\ngc}[1]{\object{NGC #1}}$
$\newcommand{\dd}[0]{ \text{d}}$
$\newcommand{\hcnthree}[0]{HCN(3--2)\xspace}$
$\newcommand{\hcnfour}[0]{HCN(4--3)\xspace}$
$\newcommand{\hcopthree}[0]{HCO^+(3--2)\xspace}$
$\newcommand{\hcopfour}[0]{HCO^+(4--3)\xspace}$
$\newcommand{\hcnone}[0]{HCN(1--0)\xspace}$
$\newcommand{\hcopone}[0]{HCO^+(1--0)\xspace}$
$\newcommand{\cotwo}[0]{CO(2--1)\xspace}$
$\newcommand{\coone}[0]{CO(1--0)\xspace}$
$\newcommand{\tkin}[0]{T_\text{K}}$
$\newcommand{\mach}[0]{\mathcal{M}}$
$\newcommand{\alphapl}[0]{\alpha_\text{PL}}$
$\newcommand{\meann}[0]{\bar{n}}$
$\newcommand{\vgrad}[0]{\nabla v}$
$\newcommand{\sstar}[0]{\Sigma_\textrm{star}}$
$\newcommand{\ssfr}[0]{\Sigma_\text{SFR}}$
$\newcommand{\meanco}[0]{\langle I_\text{CO}\rangle}$
$\newcommand{\acotwo}[0]{\alpha_\text{CO21}}$
$\newcommand{\ahcnone}[0]{\alpha_\text{HCN10}^\text{dense}}$
$\newcommand{\ahcnthree}[0]{\alpha_\text{HCN32}^\text{dense}}$
$\newcommand{\fdense}[0]{f_\text{dense}}$
$\newcommand{\kms}[0]{km~s^{-1}}$
$\newcommand{\ltom}[0]{M_\odot~(K~km~s^{-1}~pc^{2})^{-1}}$
$\newcommand{\cmmcube}[0]{cm^{-3}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\figcita$}[1]{Fig. \ref{#1}}$
$\newcommand{$\tabcita$}[1]{Table \ref{#1}}$
$\newcommand{$\refe$}[1]{{\hypersetup{hidelinks}\hyperref[fig_point]{\color{blue}#1}}}$
$\newcommand{$\ngc$}[1]{$\object${NGC #1}}$
$\newcommand{$\dd$}[0]{ \text{d}}$
$\newcommand{$\hcnthree$}[0]{HCN(3--2)$\xspace$}$
$\newcommand{$\hcnfour$}[0]{HCN(4--3)$\xspace$}$
$\newcommand{$\hcopthree$}[0]{HCO^+(3--2)$\xspace$}$
$\newcommand{$\hcopfour$}[0]{HCO^+(4--3)$\xspace$}$
$\newcommand{$\hcnone$}[0]{HCN(1--0)$\xspace$}$
$\newcommand{$\hcopone$}[0]{HCO^+(1--0)$\xspace$}$
$\newcommand{$\cotwo$}[0]{CO(2--1)$\xspace$}$
$\newcommand{$\coone$}[0]{CO(1--0)$\xspace$}$
$\newcommand{$\tkin$}[0]{T_\text{K}}$
$\newcommand{$\mach$}[0]{\mathcal{M}}$
$\newcommand{$\alphapl$}[0]{\alpha_\text{PL}}$
$\newcommand{$\meann$}[0]{\bar{n}}$
$\newcommand{$\vgrad$}[0]{\nabla v}$
$\newcommand{$\sstar$}[0]{\Sigma_\textrm{star}}$
$\newcommand{$\ssfr$}[0]{\Sigma_\text{SFR}}$
$\newcommand{$\meanco$}[0]{\langle I_\text{CO}\rangle}$
$\newcommand{$\acotwo$}[0]{\alpha_\text{CO21}}$
$\newcommand{$\ahcnone$}[0]{\alpha_\text{HCN10}^\text{dense}}$
$\newcommand{$\ahcnthree$}[0]{\alpha_\text{HCN32}^\text{dense}}$
$\newcommand{$\fdense$}[0]{f_\text{dense}}$
$\newcommand{$\kms$}[0]{km~s^{-1}}$
$\newcommand{$\ltom$}[0]{M_\odot~(K~km~s^{-1}~pc^{2})^{-1}}$
$\newcommand{$\cmmcube$}[0]{cm^{-3}}$</div>



<div id="title">

# =3-2 in nearby star-forming galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.00450-b31b1b.svg)](https://arxiv.org/abs/2302.00450)<mark>Appeared on: 2023-02-02</mark> - _Accepted for publication in A&A. 14 pages, 8 figures_

</div>
<div id="authors">

Axel Garcia-Rodriguez, et al. -- incl., <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Daizhong Liu</mark></mark>, <mark><mark>Eva Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present new HCN and HCO$^+$($J$=3--2) images of the nearby star-forming galaxies (SFGs)$\ngc${3351},$\ngc${3627}, and$\ngc${4321}. The observations, obtained with the Morita ALMA Compact Array, have a spatial resolution of$\sim$290--440 pc and resolve the inner$R_\textrm{gal} \lesssim$0.6-1 kpc of the targets, as well as the southern bar end of$\ngc${3627}. We complement this data set with publicly available images of lower excitation lines of HCN, HCO$^+$, and CO and analyse the behaviour of a representative set of line ratios:$\hcnthree$/$\hcnone$,$\hcnthree$/$\hcopthree$,$\hcnone$/$\cotwo$, and$\hcnthree$/$\cotwo$. Most of these ratios peak at the galaxy centres and decrease outwards. We compare the HCN and HCO$^+$observations with a grid of one-phase, non-local thermodynamic equilibrium (non-LTE) radiative transfer models and find them compatible with models that predict subthermally excited and optically thick lines. We study the systematic variations of the line ratios across the targets as a function of the stellar surface density ($\sstar$), the intensity-weighted$\cotwo$($\meanco$), and the star formation rate surface density ($\ssfr$). We find no apparent correlation with$\ssfr$, but positive correlations with the other two parameters, which are stronger in the case of$\meanco$. The HCN/CO-$\meanco$relations show$\lesssim$0.3 dex galaxy-to-galaxy offsets, with$\hcnthree$/$\cotwo$-$\meanco$being$\sim$2 times steeper than$\hcnone$/$\cotwo$. In contrast, the$\hcnthree$/$\hcnone$-$\meanco$relation exhibits a tighter alignment between galaxies. We conclude that the overall behaviour of the line ratios cannot be ascribed to variations in a single excitation parameter (e.g. density or temperature).

</div>

<div id="div_fig1">

<img src="tmp_2302.00450/./ngc3351-mom0_HCN32.png" alt="Fig2.1" width="11%"/><img src="tmp_2302.00450/./ngc3627-mom0_HCN32.png" alt="Fig2.2" width="11%"/><img src="tmp_2302.00450/./ngc4321-mom0_HCN32.png" alt="Fig2.3" width="11%"/><img src="tmp_2302.00450/./ngc3351-mom0_HCO+32.png" alt="Fig2.4" width="11%"/><img src="tmp_2302.00450/./ngc3627-mom0_HCO+32.png" alt="Fig2.5" width="11%"/><img src="tmp_2302.00450/./ngc4321-mom0_HCO+32.png" alt="Fig2.6" width="11%"/><img src="tmp_2302.00450/./ngc3351-strict_mom0_co21-high_res.png" alt="Fig2.7" width="11%"/><img src="tmp_2302.00450/./ngc3627-strict_mom0_co21-high_res.png" alt="Fig2.8" width="11%"/><img src="tmp_2302.00450/./ngc4321-strict_mom0_co21-high_res.png" alt="Fig2.9" width="11%"/>

**Figure 2. -** 
Integrated intensity maps at $6"$ resolution of $\ngc${3351}, $\ngc${3627}, and $\ngc${4321 from left to right}. Top and middle rows: $\hcnthree$ and $\hcopthree$  maps. The white contours correspond to S/N=5 and the grey circles represent the $6"$ beam. Offsets are relative to the positions in Table \ref{tab_sample}. Bottom row: PHANGS-ALMA $\cotwo$ maps. Black contours correspond to 10, 30, 50, and 70\% of the maximum in each $\cotwo$ image at the native ${\sim}1.5"$ resolution. The white circles indicate the fields of the $\hcnthree$ and $\hcopthree$ observations with the ACA. (*fig_mom0-HCN32*)

</div>
<div id="div_fig2">

<img src="tmp_2302.00450/./trends_all_galaxies-line_ratios.png" alt="Fig6" width="100%"/>

**Figure 6. -** Same line ratios as in Fig. \ref{fig_lineRatioMaps} plotted against $\sstar$(left), $\meanco$(middle), and $\ssfr$(right) on a pixel-by-pixel basis (grey points).
All panels have the same vertical and horizontal span in dex for an easier comparison.
For each line ratio and galaxy, the colour squares and vertical bars represent the  mean  and the $\pm 1\sigma$ standard deviation within bins of the x-axis parameter. The squares are white-filled where the mean value does not necessarily represent the whole galaxy (Sect. \ref{sec_empirical-relations}). (*fig_line-ratio_galaxy_comparison*)

</div>
<div id="div_fig3">

<img src="tmp_2302.00450/./histogram_ratios.png" alt="Fig4" width="100%"/>

**Figure 4. -** Histograms per galaxy of $\hcnthree$/$\hcopthree$, $\hcnone$/$\hcopone$, $\hcnthree$/$\hcnone$, and $\hcopthree$/$\hcopone$ line ratios. (*fig_litHist-simple*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

188  publications in the last 7 days.
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers